# Imports & Globals

In [1]:
import pickle
import pandas as pd
import numpy as np
from yaml import safe_load
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from StringProcessing import preprocessInput, encode_pad_tag, balance_complete_set

In [3]:
from SiameseModel import CosineDistance, ContrastiveLoss 

tf.version:  2.9.1
tf.keras.version:  2.9.0
tf.config.devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2022-10-21 13:02:42.698498: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-21 13:02:42.721978: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-21 13:02:42.722137: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-21 13:02:42.723421: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

In [4]:
from ModelEvaluation import find_Model_ranks

In [5]:
#Maximum sequence length including padding
global MAXLEN
with open('LSTM65.yaml', 'r') as file:
    model_config = safe_load(file)
    MAXLEN = model_config['MAXLEN']

# Pickled Datasets

In [6]:
dUnique_df = pd.read_pickle("../data/dUnique_df.pkl")

--------------------------------

# Build model, load weights and evaluate on test data

In [7]:
model = tf.keras.models.load_model(
    './saved_models/{}'.format(model_config["Model_Name"]), 
    custom_objects = {
        'CosineDistance': CosineDistance,
        'ContrastiveLoss': ContrastiveLoss
    }, 
    compile=True, 
    options=None
)
model.summary()

2022-10-21 13:02:43.197053: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-21 13:02:43.197237: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-21 13:02:43.197343: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-21 13:02:43.197478: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-21 13:02:43.197584: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

2022-10-21 13:02:49.550576: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-10-21 13:02:49.598430: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-10-21 13:02:49.608329: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-10-21 13:02:49.646446: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-10-21 13:02:49.765196: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond/while' has 13 outputs but the _output_shapes at

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 65)]         0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 65)]         0           []                               
                                                                                                  
 model (Functional)             (None, 65)           76425       ['input_2[0][0]',                
                                                                  'input_3[0][0]']                
                                                                                                  
 lambda (Lambda)                (None, 1)            0           ['model[0][0]',            

---------------------------------------------------------------

# Evaluation

## Unique target labels

In [8]:
dUnique_df.head()

,dUnique_label,dUnique_seq_padded
0,AAI,"[1, 1, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,AARONSROD,"[1, 1, 18, 15, 14, 19, 18, 15, 4, 0, 0, 0, 0, ..."
2,ABIES ALBA,"[1, 2, 9, 5, 19, 27, 1, 12, 2, 1, 0, 0, 0, 0, ..."
3,ABSINTHE,"[1, 2, 19, 9, 14, 20, 8, 5, 0, 0, 0, 0, 0, 0, ..."
4,ABSINTHE WORMWOOD,"[1, 2, 19, 9, 14, 20, 8, 5, 27, 23, 15, 18, 13..."


In [9]:
dUnique_df.index.size

1879

In [10]:
latin_2_common = preprocessInput('../data/vocabulary_LB_to_common_names_09082022.tsv', maxlen= MAXLEN, reflexive=True, balance = False, noise = None, sep='\t')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1299 entries, 0 to 1298
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   lb        1299 non-null   object
 1   mappings  1299 non-null   object
dtypes: object(2)
memory usage: 20.4+ KB
None
Processing file: ----------------------------------------
Renaming colums:
	Current names: Index(['lb', 'mappings'], dtype='object')
	New names: Index(['x', 'y'], dtype='object')
Dropping row with empty cells:
	Dropped 0 rows with empty cells.
	Cleaning string sequences.
	Uppercasing string sequences.
Dropping sequences longer than the maxlen of 65:
	Dropped 0 that exceeded the maximum sequence length.
Dropping duplicate sequences:
	Dropped 321 duplicate sequences.
Done processing: ---------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2277 entries, 0 to 2276
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------

In [11]:
common_2_latin = preprocessInput('../data/positive-unmapped-pairs-common-name-or-latin-binomial-copies.tsv', maxlen= MAXLEN, reflexive=True, balance = False, noise = None, sep='\t')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728 entries, 0 to 727
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   common_name     728 non-null    object
 1   latin_binomial  728 non-null    object
dtypes: object(2)
memory usage: 11.5+ KB
None
Processing file: ----------------------------------------
Renaming colums:
	Current names: Index(['common_name', 'latin_binomial'], dtype='object')
	New names: Index(['x', 'y'], dtype='object')
Dropping row with empty cells:
	Dropped 0 rows with empty cells.
	Cleaning string sequences.
	Uppercasing string sequences.
Dropping sequences longer than the maxlen of 65:
	Dropped 0 that exceeded the maximum sequence length.
Dropping duplicate sequences:
	Dropped 728 duplicate sequences.
Done processing: ---------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728 entries, 0 to 727
Data columns (total 2 columns):
 #   Column  Non-Null Count 

In [12]:
gsrs_2_common = preprocessInput('../data/positive-unmapped-pairs-gsrs-name-to-common-name.tsv', maxlen= MAXLEN, reflexive=True, balance = False, noise = None, sep='\t')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10838 entries, 0 to 10837
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         10838 non-null  object
 1   common_name  199 non-null    object
dtypes: object(2)
memory usage: 169.5+ KB
None
Processing file: ----------------------------------------
Renaming colums:
	Current names: Index(['name', 'common_name'], dtype='object')
	New names: Index(['x', 'y'], dtype='object')
Dropping row with empty cells:
	Dropped 10639 rows with empty cells.
	Cleaning string sequences.
	Uppercasing string sequences.
Dropping sequences longer than the maxlen of 65:
	Dropped 0 that exceeded the maximum sequence length.
Dropping duplicate sequences:
	Dropped 0 duplicate sequences.
Done processing: ---------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  --

In [13]:
# vocab = pd.read_csv('../data/lb_to_common_names.csv')
# vocab = vocab[['latin_binomial_clean','common_name_clean']].rename(columns={'latin_binomial_clean': "x", 'common_name_clean': "y"})
# vocab.info()

In [14]:
equivalence_set = pd.concat([latin_2_common, common_2_latin, gsrs_2_common], ignore_index=True)
equivalence_set.drop_duplicates(subset=['x', 'y'], inplace=True)

In [15]:
equivalence_set[equivalence_set["x"] == "CRANBERRY"]

,x,y
2230,CRANBERRY,VACCINIUM MACROCARPON


## True Positives

In [16]:
matches = encode_pad_tag(preprocessInput('../data/NP_FAERS_mapped_20220215.csv', maxlen=MAXLEN, reflexive=False, noise=None, balance=False), match=1, distance=0, maxlen=MAXLEN)
matches.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5358 entries, 0 to 5357
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   FAERS_drug_match  5358 non-null   object
 1   lookup_value      5358 non-null   object
dtypes: object(2)
memory usage: 83.8+ KB
None
Processing file: ----------------------------------------
Renaming colums:
	Current names: Index(['FAERS_drug_match', 'lookup_value'], dtype='object')
	New names: Index(['x', 'y'], dtype='object')
Dropping row with empty cells:
	Dropped 0 rows with empty cells.
	Cleaning string sequences.
	Uppercasing string sequences.
Dropping sequences longer than the maxlen of 65:
	Dropped 256 that exceeded the maximum sequence length.
Dropping duplicate sequences:
	Dropped 487 duplicate sequences.
Done processing: ---------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4615 entries, 0 to 4614
Data columns (total 2 columns):
 #   Column

,x,y,Processed_x,Processed_y,Match,Distance
0,ASHWAGANDHA,ASHWAGANDA,"[1, 19, 8, 23, 1, 7, 1, 14, 4, 8, 1, 0, 0, 0, ...","[1, 19, 8, 23, 1, 7, 1, 14, 4, 1, 0, 0, 0, 0, ...",1,0
1,ASHWAGANDHA CATS CLAW PAU DARCO RHODIOLA STEMM...,ASHWAGANDA,"[1, 19, 8, 23, 1, 7, 1, 14, 4, 8, 1, 27, 3, 1,...","[1, 19, 8, 23, 1, 7, 1, 14, 4, 1, 0, 0, 0, 0, ...",1,0
2,ASHWAGANDHA EXTRACT,ASHWAGANDA,"[1, 19, 8, 23, 1, 7, 1, 14, 4, 8, 1, 27, 5, 24...","[1, 19, 8, 23, 1, 7, 1, 14, 4, 1, 0, 0, 0, 0, ...",1,0
3,ASHWAGANDHA ROOT EXTRACT,ASHWAGANDA,"[1, 19, 8, 23, 1, 7, 1, 14, 4, 8, 1, 27, 18, 1...","[1, 19, 8, 23, 1, 7, 1, 14, 4, 1, 0, 0, 0, 0, ...",1,0
4,ASHWAGANDHA WITHANIA SOMNIFERA,ASHWAGANDA,"[1, 19, 8, 23, 1, 7, 1, 14, 4, 8, 1, 27, 23, 9...","[1, 19, 8, 23, 1, 7, 1, 14, 4, 1, 0, 0, 0, 0, ...",1,0


### ASHWAGANDA

In [17]:
matches.loc[1, "x"]

'ASHWAGANDHA CATS CLAW PAU DARCO RHODIOLA STEMMACANTHA'

In [18]:
predicts = model.predict([np.tile(matches.loc[1, "Processed_x"], (dUnique_df.index.size,1)), np.stack(dUnique_df['dUnique_seq_padded'])]).astype('float32')
top5 = predicts.flatten().argsort()

2022-10-21 13:02:53.910826: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1750] (One-time warning): Not using XLA:CPU for cluster.

If you want XLA:CPU, do one of the following:

 - set the TF_XLA_FLAGS to include "--tf_xla_cpu_global_jit", or
 - set cpu_global_jit to true on this session's OptimizerOptions, or
 - use experimental_jit_scope, or
 - use tf.function(jit_compile=True).

To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a
proper command-line flag, not via TF_XLA_FLAGS).
2022-10-21 13:02:56.257439: I tensorflow/compiler/xla/service/service.cc:170] XLA service 0x7fec90004cd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-10-21 13:02:56.257459: I tensorflow/compiler/xla/service/service.cc:178]   StreamExecutor device (0): NVIDIA GeForce RTX 2070 with Max-Q Design, Compute Capability 7.5
2022-10-21 13:02:56.260939: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:263] disabling MLIR cras

59/59 [==============================] - 4s 4ms/step


In [19]:
# Top-5 smalles distances
{
    "rank1": dUnique_df.iloc[top5[0]]['dUnique_label'],
    "rank2": dUnique_df.iloc[top5[1]]['dUnique_label'],
    "rank3": dUnique_df.iloc[top5[2]]['dUnique_label'],
    "rank4": dUnique_df.iloc[top5[3]]['dUnique_label'],
    "rank5": dUnique_df.iloc[top5[4]]['dUnique_label'],
}

{'rank1': 'MEDICAGO POLYMORPHA',
 'rank2': 'SICHUAN PAGODATREE',
 'rank3': 'HELMINTHOTHECA ECHIOIDES',
 'rank4': 'LAPACHO NEGRO',
 'rank5': 'ASHWAGANDHA'}

### Echinacea

In [21]:
Echinacea = matches[matches["y"].str.contains("ECHINACEA")].head(1)
Echinacea

,x,y,Processed_x,Processed_y,Match,Distance
949,ASCORBIC ACID ECHINACEA PURPUREA ZINC CITRATE ...,ECHINACEA,"[1, 19, 3, 15, 18, 2, 9, 3, 27, 1, 3, 9, 4, 27...","[5, 3, 8, 9, 14, 1, 3, 5, 1, 0, 0, 0, 0, 0, 0,...",1,0


In [25]:
i = Echinacea.index.values[0]
matches["x"][i]

'ASCORBIC ACID ECHINACEA PURPUREA ZINC CITRATE ZINC GLUCONATE'

In [26]:
predicts = model.predict([np.tile(matches.loc[i, "Processed_x"], (dUnique_df.index.size,1)), np.stack(dUnique_df['dUnique_seq_padded'])]).astype('float32')
top5 = predicts.flatten().argsort()

59/59 [==============================] - 1s 11ms/step


In [27]:
# Top-5 smalles distances
m = max(top5)
{
    "rank1": dUnique_df.iloc[top5[0]]['dUnique_label'],
    "rank2": dUnique_df.iloc[top5[1]]['dUnique_label'],
    "rank3": dUnique_df.iloc[top5[2]]['dUnique_label'],
    "rank4": dUnique_df.iloc[top5[3]]['dUnique_label'],
    "rank5": dUnique_df.iloc[top5[4]]['dUnique_label'],
}

{'rank1': 'CICHORIUM INTYBUS',
 'rank2': 'SMALL CENTAURY',
 'rank3': 'AMERICANBUGLEWEED',
 'rank4': 'CHINESE CINNAMON TREE',
 'rank5': 'HAG TAPER'}

### Cranberry

In [28]:
cranberry = dUnique_df[dUnique_df["dUnique_label"].str.contains("CRANBERRY")]
cranberry

,dUnique_label,dUnique_seq_padded
510,CRANBERRY,"[3, 18, 1, 14, 2, 5, 18, 18, 25, 0, 0, 0, 0, 0..."


In [29]:
i = cranberry.index
dUnique_df.loc[i, "dUnique_label"]

510    CRANBERRY
Name: dUnique_label, dtype: object

In [30]:
predicts = model.predict([
    np.tile(dUnique_df.loc[i, "dUnique_seq_padded"].to_list(), (dUnique_df.index.size, 1)),
    np.stack(dUnique_df['dUnique_seq_padded'])
]).astype('float32')

top5 = predicts.flatten().argsort()

59/59 [==============================] - 0s 2ms/step


In [31]:
# Top-5 smalles distances
m = max(top5)
{
    "rank1": dUnique_df.iloc[top5[0]]['dUnique_label'],
    "rank2": dUnique_df.iloc[top5[1]]['dUnique_label'],
    "rank3": dUnique_df.iloc[top5[2]]['dUnique_label'],
    "rank4": dUnique_df.iloc[top5[3]]['dUnique_label'],
    "rank5": dUnique_df.iloc[top5[4]]['dUnique_label'],
}

{'rank1': 'CRANBERRY',
 'rank2': 'VACCINIUM MACROCARPON',
 'rank3': 'WINTERCHERRY',
 'rank4': 'CANDLEBERRY',
 'rank5': 'WINTER SAVORY'}

## True  Negatives

In [32]:
find_Model_ranks(model, df=matches.sample(20), targets= dUnique_df, equivalence_set=equivalence_set, ranks=5, find_related_rank=False, report_distances=False)

,x,y,Processed_x,Processed_y,Match,Distance,rank1,rank2,rank3,rank4,rank5
1468,BLACK COHESH,BLACK COHOSH,"[2, 12, 1, 3, 11, 27, 3, 15, 8, 5, 19, 8, 0, 0...","[2, 12, 1, 3, 11, 27, 3, 15, 8, 15, 19, 8, 0, ...",1,0,BLACK COHOSH,BLUE COHOSH,ACTAEA RACEMOSA,BLACK SNAKEROOT,ALCEA SETOSA
3364,SAW PALMETTO STANDARDIZEL SERENOA REPENS,SERENOA REPENS,"[19, 1, 23, 27, 16, 1, 12, 13, 5, 20, 20, 15, ...","[19, 5, 18, 5, 14, 15, 1, 27, 18, 5, 16, 5, 14...",1,0,SERENOA REPENS,SAW PALMETTO,ASCLEPIAS PERENNIS,SAWPALMETTO,ST BARTHOLOMEWS TEA
1174,CRATAEUGUTT CRATAEGUS LAEVIGATA EXTRACT,WOODLAND HAWTHORN,"[3, 18, 1, 20, 1, 5, 21, 7, 21, 20, 20, 27, 3,...","[23, 15, 15, 4, 12, 1, 14, 4, 27, 8, 1, 23, 20...",1,0,CRATAEGUS LAEVIGATA,RUSCUS ACULEATUS,CITRUS RETICULATA,TREETURMERIC,WOODLAND HAWTHORN
1793,BEDFISH OIL CRANBERRY,CRANBERRY,"[2, 5, 4, 6, 9, 19, 8, 27, 15, 9, 12, 27, 3, 1...","[3, 18, 1, 14, 2, 5, 18, 18, 25, 0, 0, 0, 0, 0...",1,0,CANDLEBERRY,JAUNDICEBERRY,BERBERIS VULGARIS,COMMON BARBERRY,BEEBREAD
1685,APPLE CIDAR VINEGAR SUPPLEMENTS,APPLE CIDER VINEGAR,"[1, 16, 16, 12, 5, 27, 3, 9, 4, 1, 18, 27, 22,...","[1, 16, 16, 12, 5, 27, 3, 9, 4, 5, 18, 27, 22,...",1,0,SLIPPERYROOT,AMERICANBUGLEWEED,AMERICAN SPIKENARD,SPOONLEAF YUCCA,HARPAGOPHYTUM PROCUMBENS
3065,THC MARIJUANA,CANNABIS SATIVA,"[20, 8, 3, 27, 13, 1, 18, 9, 10, 21, 1, 14, 1,...","[3, 1, 14, 14, 1, 2, 9, 19, 27, 19, 1, 20, 9, ...",1,0,MARIJUANA,YI ZHU QIAN MA,DAMIANA,HE HUAN,CLITORIA MARIANA
1208,ASHWAGAN,ASHWAGANDA,"[1, 19, 8, 23, 1, 7, 1, 14, 0, 0, 0, 0, 0, 0, ...","[1, 19, 8, 23, 1, 7, 1, 14, 4, 1, 0, 0, 0, 0, ...",1,0,WITHANIA SOMNIFERA,ASHWAGANDHA,ASHWAGANDA,WINTERCHERRY,ASIAN GINSENG
2222,GARLIC PEPPER,GARLIC,"[7, 1, 18, 12, 9, 3, 27, 16, 5, 16, 16, 5, 18,...","[7, 1, 18, 12, 9, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0...",1,0,GUINEAPEPPER,BLACK PEPPER,KAWA PEPPER,GALIUM VERUM,WHITE PEPPER
355,FLAX SEE POWDER AND OIL,FLAX SEED,"[6, 12, 1, 24, 27, 19, 5, 5, 27, 16, 15, 23, 4...","[6, 12, 1, 24, 27, 19, 5, 5, 4, 0, 0, 0, 0, 0,...",1,0,HORNY GOAT WEED,MEADOWSWEET,EPIMEDIUM PUBESCENS MAXIM,BLACK SAMPSON,EASTERN PURPLECONEFLOWER
2640,FLAXSEED OIL PILL,LINUM USITATISSIMUM,"[6, 12, 1, 24, 19, 5, 5, 4, 27, 15, 9, 12, 27,...","[12, 9, 14, 21, 13, 27, 21, 19, 9, 20, 1, 20, ...",1,0,DILL,NYMPHAEA ODORATA,FILIPENDULA ULMARIA,GARDEN DILL,MEADOWSWEET


In [33]:
subsample = dUnique_df.sample(20)
subsample.rename(columns={"dUnique_seq_padded": 'Processed_dUnique_label'}, inplace=True)

In [34]:
sanity = find_Model_ranks(model, df=subsample, targets= dUnique_df, equivalence_set=equivalence_set, ranks=5, find_related_rank=False, report_distances=False)

In [35]:
sanity

,dUnique_label,Processed_dUnique_label,rank1,rank2,rank3,rank4,rank5
1361,QUARTER BONNET,"[17, 21, 1, 18, 20, 5, 18, 27, 2, 15, 14, 14, ...",QUARTER BONNET,SUGAR BEET,SCUTELLARIA LATERIFLORA,MADDOG SCULLCAP,PURPLE BETONY
608,EMILIA SONCHIFOLIA,"[5, 13, 9, 12, 9, 1, 27, 19, 15, 14, 3, 8, 9, ...",EMILIA SONCHIFOLIA,EMILIA,CUPIDS SHAVINGBRUSH,SOW THISTLE,MELISSA
1637,ST MARYS THISTLE,"[19, 20, 27, 13, 1, 18, 25, 19, 27, 20, 8, 9, ...",ST MARYS THISTLE,MARYS THISTLE,LADYS THISTLE,MARY THISTLE,SOW THISTLE
1674,SWERTIA CHIRATA,"[19, 23, 5, 18, 20, 9, 1, 27, 3, 8, 9, 18, 1, ...",SWERTIA CHIRATA,CHIRATA,CHIRETTA,KIRATATIKTA,SPERMACOCE VERTICILLATA
1628,SQUAW BALM,"[19, 17, 21, 1, 23, 27, 2, 1, 12, 13, 0, 0, 0,...",SQUAW BALM,ASSAI PALM,MENTHA PULEGIUM,MOSQUITO PLANT,UVA URSI
78,ANNATTO,"[1, 14, 14, 1, 20, 20, 15, 0, 0, 0, 0, 0, 0, 0...",ANNATTO,ARNATTO,BIXA ORELLANA,XIAO MAI,NEUROLAENA LOBATA
1742,TRUE CHAMOMILE,"[20, 18, 21, 5, 27, 3, 8, 1, 13, 15, 13, 9, 12...",TRUE CHAMOMILE,GERMAN CHAMOMILE,BLUE CHAMOMILE,MATRICARIA CHAMOMILLA,CHAMOMILE
1554,SHATAVARI,"[19, 8, 1, 20, 1, 22, 1, 18, 9, 0, 0, 0, 0, 0,...",SHATAVARI,SHATAMULI,ASPARAGUS RACEMOSUS,SWEET CALAMUS,SPINACH BEET
110,ASCLEPIAS PERENNIS,"[1, 19, 3, 12, 5, 16, 9, 1, 19, 27, 16, 5, 18,...",ASCLEPIAS PERENNIS,SERENOA REPENS,LUPINUS PERENNIS,ASCLEPIAS STELLIFERA,SCRUBPALMETTO
1158,MOUNTAINGRAPE,"[13, 15, 21, 14, 20, 1, 9, 14, 7, 18, 1, 16, 5...",MOUNTAINGRAPE,MOUNTAIN GRAPE,OREGON GRAPE,OREGONGRAPE,MUSCADINE GRAPE
